## EfficientNet

In [19]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as T
import numpy as np
import os
import copy
from cv2 import cv2
import matplotlib.pyplot as plt

from torchvision.transforms import Compose, Resize, ToTensor
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.optim import lr_scheduler
from torch import Tensor
from torchsummary import summary
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# custom module
from new.new_dataset import CustomDataset
from new.new_utils import is_val_loss_decreasing, check_image, check_loader, get_transformer
from new.new_model import EnsembleModel, MaskModel
from new.new_inference import test_prediction
from new.new_dataset import CustomDataset, MaskDataset

In [13]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b2')

Loaded pretrained weights for efficientnet-b2


In [16]:
# freeze
for n, p in model.named_parameters():
    if '_fc' not in n:
        p.requires_grad = False

In [8]:
root = '../input/data'

transform = get_transformer(False)
train_ds = CustomDataset(root, transform['origin'], category='age')
test_ds = MaskDataset(root, transform['origin'], train=False)

train, val = train_test_split(train_ds, test_size=0.1, shuffle=True, random_state=43)

batch_size = 128
loaders = {
    'train': DataLoader(train, batch_size=128, num_workers=4, pin_memory=True, drop_last=True),
    'val': DataLoader(val, batch_size=128, num_workers=4, pin_memory=True, drop_last=True),
    'test': DataLoader(test_ds, batch_size=128, num_workers=4, pin_memory=True)
}

In [22]:
print('=============== Start Training ===============')
    
device = 'cuda'
num_epochs = 10

# ============================
# for viz
# ============================
losses = {'train':[], 'val':[]}
accuracies = {'train':[], 'val':[]}
lr = []

model.to(device)
criterion = nn.CrossEntropyLoss().to(device)

# ============================
# optimizers
# ============================
# optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
# optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001, eps=1e-08)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

# ============================
# schedulers
# ============================
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 0.1, epochs=epochs, steps_per_epoch=len(loaders['train']), cycle_momentum=True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 3, gamma=0.5)

# time tracking
since = time.time()

# init
best_model = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0.0
        total = 0.0

        for inputs, labels in tqdm(loaders[phase]):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'): # back prop only training
                outp = model(inputs)
                loss = criterion(outp, labels)
                _, pred = torch.max(outp, 1)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    # scheduler.step()
                    # lr.append(scheduler.get_lr())

            running_loss += loss.item() * inputs.size(0)            # per batch_size
            running_corrects += torch.sum(pred == labels.data)      # per batch_size
            total += labels.size(0)

        if phase == 'train':
            acc = 100. * running_corrects.double() / total
            scheduler.step(acc)

        epoch_loss = running_loss / total                        # per epoch
        epoch_acc = 100. * running_corrects.double() / total     # per epoch

        losses[phase].append(epoch_loss)
        accuracies[phase].append(epoch_acc)

        if phase == 'train':
            print('Epoch : {} / {}'.format(epoch + 1, num_epochs))
        print('{} - Loss : {:.4f}, Acc : {:.4f}%'.format(phase, epoch_loss, epoch_acc))
        lr.append(scheduler._last_lr)

        if phase == 'val':
            print('Training Time Spent : {}m {:.4f}s'.format((time.time() - since) // 60, (time.time() - since) % 60))

        # update best result
        if phase == 'val' and epoch_acc > best_acc:
            print('--- Update Best Model ---')
            best_acc = epoch_acc
            best_model = copy.deepcopy(model.state_dict())

        # dividing line
        print('==' * 15)

    # early stopping
#     if not is_val_loss_decreasing(epoch, 3, losses['val']):
#         print('--- Early Stopping ---')
#         break

time_elapsed = time.time() - since
print('Whole Time Spent : {}m {:.4f}s'.format(time_elapsed // 60, time_elapsed % 60))

# load best model
model.load_state_dict(best_model)

# save model
root = '../code/model'
torch.save(model, os.path.join(root, 'efficientnetb2.pth'))

  0%|          | 0/132 [00:00<?, ?it/s]

=============== Start Training ===============


100%|██████████| 132/132 [00:17<00:00,  7.60it/s]
/opt/conda/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
  0%|          | 0/14 [00:00<?, ?it/s]

Epoch : 1 / 10
train - Loss : 0.5350, Acc : 83.1499%


  0%|          | 0/132 [00:00<?, ?it/s]

...

val - Loss : 0.4956, Acc : 84.3750%
Training Time Spent : 3.0m 5.2895s


  0%|          | 0/14 [00:00<?, ?it/s]

Epoch : 10 / 10
train - Loss : 0.4805, Acc : 84.2448%


100%|██████████| 14/14 [00:03<00:00,  4.61it/s]

val - Loss : 0.4979, Acc : 84.3750%
Training Time Spent : 3.0m 26.1544s
Whole Time Spent : 3.0m 26.1548s


In [24]:
all_predictions = []
model.eval()
with torch.no_grad():
    for images, labels in tqdm(loaders['test']):
        images = images.to(device)
        labels = labels.to(device)

        outp = model(images)
        tmp_outp = outp.argmax(dim=-1) # index 1 => attention info
        all_predictions.extend(tmp_outp.cpu().numpy())

test_ds.info_df['ans'] = all_predictions
test_ds.info_df.to_csv('../input/data/eval/submission_efficientb2.csv', index=False)
print('================= Done =================')

100%|██████████| 99/99 [00:22<00:00,  4.32it/s]

================= Done =================
